<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [9]:
# !pip3 install folium
# !pip3 install wget
# !pip3 install pandas

In [10]:
import folium
import wget
import pandas as pd

In [11]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [17]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [19]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df
v=spacex_df['Launch Site']
v.unique 

<bound method Series.unique of 0      CCAFS LC-40
1      CCAFS LC-40
2      CCAFS LC-40
3      CCAFS LC-40
4      CCAFS LC-40
5      CCAFS LC-40
6      CCAFS LC-40
7      CCAFS LC-40
8      CCAFS LC-40
9      CCAFS LC-40
10     CCAFS LC-40
11     CCAFS LC-40
12     CCAFS LC-40
13     CCAFS LC-40
14     CCAFS LC-40
15     CCAFS LC-40
16     CCAFS LC-40
17     CCAFS LC-40
18     CCAFS LC-40
19     CCAFS LC-40
20     CCAFS LC-40
21     CCAFS LC-40
22     CCAFS LC-40
23     CCAFS LC-40
24     CCAFS LC-40
25     CCAFS LC-40
26     VAFB SLC-4E
27     VAFB SLC-4E
28     VAFB SLC-4E
29     VAFB SLC-4E
30     VAFB SLC-4E
31     VAFB SLC-4E
32     VAFB SLC-4E
33     VAFB SLC-4E
34     VAFB SLC-4E
35     VAFB SLC-4E
36      KSC LC-39A
37      KSC LC-39A
38      KSC LC-39A
39      KSC LC-39A
40      KSC LC-39A
41      KSC LC-39A
42      KSC LC-39A
43      KSC LC-39A
44      KSC LC-39A
45      KSC LC-39A
46      KSC LC-39A
47      KSC LC-39A
48      KSC LC-39A
49    CCAFS SLC-40
50    CCAFS SLC-40


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [22]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [24]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [25]:
# Create a circle and marker for NASA Johnson Space Center
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)
# Loop through each launch site and add a Circle and Marker to the map
for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    # Add a circle at each launch site's coordinate
    folium.Circle(
        site_coordinate,
        radius=1000,
        color='#0000FF',
        fill=True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    # Add a marker for each launch site
    folium.map.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#0000FF;"><b>%s</b></div>' % row['Launch Site'],
        )
    ).add_to(site_map)
site_map

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [33]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [39]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [43]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [45]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Step 3: Create a new column 'marker_color' based on the 'class' column
# If class=1, marker_color will be 'green' for success; if class=0, marker_color will be 'red' for failure
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Step 4: Initialize the map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Step 5: Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Step 6: Add markers for each launch record with success or failure colors
for index, row in spacex_df.iterrows():
    launch_coordinate = [row['Lat'], row['Long']]
    # Add a marker with the specified color for success or failure
    folium.Marker(
        location=launch_coordinate,
        icon=folium.Icon(color=row['marker_color']),
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Display the map
site_map

In [46]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [48]:
# Add marker_cluster to current site_map
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Step 4: Initialize the map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Step 5: Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Step 6: Add markers with customized icons to indicate success/failure
for index, record in spacex_df.iterrows():
    launch_coordinate = [record['Lat'], record['Long']]
    # Create a marker with a white background and icon color based on 'marker_color'
    marker = folium.Marker(
        location=launch_coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='ok-sign' if record['class'] == 1 else 'remove-sign'),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [56]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [59]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [61]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Define coordinates of the closest coastline (example coordinates near Kennedy Space Center)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Function to calculate distance between two latitude/longitude points
from math import radians, cos, sin, asin, sqrt

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula to calculate the great circle distance
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers (mean value)
    r = 6371.0
    return c * r

# Example coordinates for a launch site
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Calculate distance to the closest coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline


7.42699044746402

_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [63]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from folium import DivIcon

# Define the coordinates for the selected closest coastline point
coastline_lat = 28.56367
coastline_lon = -80.57163

# Example coordinates for a launch site
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Calculate the distance to the coastline (from previous code)
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium map centered at the coastline
site_map = folium.Map(location=[coastline_lat, coastline_lon], zoom_start=12)

# Create and add a marker at the coastline point
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the marker to the map
distance_marker.add_to(site_map)

# Display the map
site_map


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [65]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)

# Define the coordinates for the selected coastline point and launch site
coastline_lat = 28.56367
coastline_lon = -80.57163
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Create the map centered around the coastline
site_map = folium.Map(location=[coastline_lat, coastline_lon], zoom_start=12)

# Define the coordinates for the PolyLine (coastline to launch site)
coordinates = [(coastline_lat, coastline_lon), (launch_site_lat, launch_site_lon)]

# Create a PolyLine object to draw a line between the coastline and launch site
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [75]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
import folium
#%pip install geopy

from geopy.distance import geodesic

# Define the coordinates for the launch site and the closest city
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Example coordinates for a nearby city (you can replace this with actual data)
city_lat = 28.559805  # Approximate latitude of Cocoa Beach, FL
city_lon = -80.588315  # Approximate longitude of Cocoa Beach, FL

# Create the map centered around the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Calculate the distance from the launch site to the city
launch_site = (launch_site_lat, launch_site_lon)
city_location = (city_lat, city_lon)
distance_to_city = geodesic(launch_site, city_location).kilometers

# Add a marker for the closest city
folium.Marker(
    location=[city_lat, city_lon],
    popup=f"Closest City: Cocoa Beach\nDistance to Launch Site: {distance_to_city:.2f} km",
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Draw a line (PolyLine) between the city and the launch site
coordinates = [(city_lat, city_lon), (launch_site_lat, launch_site_lon)]
folium.PolyLine(locations=coordinates, weight=2, color='green').add_to(site_map)

# Optionally add a marker at the launch site
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color='red')
).add_to(site_map)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


In [110]:
# Install necessary libraries
# %pip install dash dash-bootstrap-components plotly pandas

import dash
%pip install dash-bootstrap-components
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

# Sample Data: Replace with your actual SpaceX launch data
data = {
    "Launch Date": ["2023-01-10", "2023-03-15", "2023-05-20", "2023-07-30", "2023-09-25"],
    "Success": [True, False, True, True, False],
    "Rocket Type": ["Falcon 9", "Falcon 9", "Falcon Heavy", "Falcon 9", "Falcon Heavy"],
}
launch_df = pd.DataFrame(data)
launch_df["Launch Date"] = pd.to_datetime(launch_df["Launch Date"])

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define app layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("SpaceX Launch Dashboard", style={'text-align': 'center'})),
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("Filter by Date Range"),
            dcc.DatePickerRange(
                id="date-range-picker",
                start_date=launch_df["Launch Date"].min(),
                end_date=launch_df["Launch Date"].max(),
                display_format="YYYY-MM-DD",
            ),
            html.Br(),
            html.Br(),
            html.Label("Filter by Rocket Type"),
            dcc.Dropdown(
                id="rocket-type-dropdown",
                options=[{"label": rt, "value": rt} for rt in launch_df["Rocket Type"].unique()],
                multi=True,
                placeholder="Select Rocket Type",
            ),
        ], width=3),
        dbc.Col([
            dcc.Graph(id="launch-success-pie-chart"),
            dcc.Graph(id="launch-frequency-line-chart"),
        ], width=9),
    ])
])

# Define callback for interactive filtering
@app.callback(
    [Output("launch-success-pie-chart", "figure"),
     Output("launch-frequency-line-chart", "figure")],
    [Input("date-range-picker", "start_date"),
     Input("date-range-picker", "end_date"),
     Input("rocket-type-dropdown", "value")]
)
def update_charts(start_date, end_date, selected_rockets):
    # Filter data based on inputs
    filtered_df = launch_df[(launch_df["Launch Date"] >= start_date) & (launch_df["Launch Date"] <= end_date)]
    if selected_rockets:
        filtered_df = filtered_df[filtered_df["Rocket Type"].isin(selected_rockets)]
    
    # Launch Success Pie Chart
    success_counts = filtered_df["Success"].value_counts().rename({True: "Success", False: "Failure"})
    pie_chart = px.pie(success_counts, values=success_counts.values, names=success_counts.index, 
                       title="Launch Success Rate")

    # Launch Frequency Line Chart
    frequency_data = filtered_df.groupby("Launch Date").size().reset_index(name="Launch Count")
    line_chart = px.line(frequency_data, x="Launch Date", y="Launch Count", 
                         title="Launch Frequency Over Time")
    
    return pie_chart, line_chart

# Run app
if __name__ == "__main__":
    app.run_server(debug=True)


   ---------------------------------------- 0.0/222.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/222.5 kB ? eta -:--:--
   ----- --------------------------------- 30.7/222.5 kB 435.7 kB/s eta 0:00:01
   ----- --------------------------------- 30.7/222.5 kB 435.7 kB/s eta 0:00:01
   ------------------------ ------------- 143.4/222.5 kB 944.1 kB/s eta 0:00:01
   ---------------------------------------- 222.5/222.5 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
